In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
%matplotlib inline

/Users/NYer510/anaconda/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')
/Users/NYer510/anaconda/lib/python2.7/site-packages/matplotlib/__init__.py:872: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


In [2]:
df_batter = pd.read_csv('../../baseball_dataset/batting2015.csv',index_col=0)
df_batter.head()

,Team,背番号,選手名,打率,試合,打席数,打数,安打,本塁打,打点,...,四球,死球,三振,犠打,併殺打,長打率,出塁率,OPS,RC27,XR27
0,広,33,菊池 涼介,0.256,142,640,558,143,8,32,...,29,2,91,49,7,0.346,0.294,0.640,3.28,3.25
1,広,9,丸 佳浩,0.250,142,629,527,132,19,63,...,93,1,142,4,4,0.416,0.362,0.777,5.60,5.63
2,広,63,田中 広輔,0.275,140,587,541,149,8,45,...,33,7,104,5,8,0.414,0.325,0.739,4.66,4.47
3,広,28,新井 貴浩,0.275,124,476,422,116,7,57,...,48,2,73,0,15,0.386,0.349,0.735,4.69,4.63
4,広,6,梵 英心,0.237,96,332,283,67,6,27,...,36,1,49,10,6,0.360,0.323,0.683,3.89,3.93


In [3]:
for c in df_batter.columns:
    print c

Team
背番号
選手名
打率
試合
打席数
打数
安打
本塁打
打点
盗塁
四球
死球
三振
犠打
併殺打
長打率
出塁率
OPS
RC27
XR27


In [4]:
df_pitcher = pd.read_csv('../../baseball_dataset/pitcher2015.csv',index_col=0)
df_pitcher.head()

,Team,背番号,選手名,防御率,試合,勝利,敗北,セlブ,ホlルド,勝率,...,投球回,被安打,被本塁打,与四球,与死球,奪三振,失点,自責点,WHIP,DIPS
0,広,18,前田 健太,2.17,28,15,8,0,0,0.652,...,199.1,162,5,38,6,170,49,48,1.00,2.40
1,広,42,ジョンソン,1.85,28,14,7,0,0,0.667,...,194.1,146,5,67,2,150,43,40,1.10,2.98
2,広,15,黒田 博樹,2.55,26,11,8,0,0,0.579,...,169.2,158,8,29,7,106,53,48,1.10,3.12
3,広,11,福井 優也,3.56,21,9,6,0,0,0.600,...,131.1,106,10,53,5,99,53,52,1.21,3.93
4,広,14,大瀬良 大地,2.89,50,3,7,2,20,0.300,...,109.0,108,5,31,1,97,50,35,1.28,2.82


In [5]:
name_j_to_e = {
    '選手名':'name',
    '打率':'ave',
    '試合':'on_games',
    '打席数':'ab',
    '打数':'pa',
    '安打':'hit',
    '本塁打':'hr',
    '打点':'bp',
    '四球':'walk',
    '死球':'deadb',
    '三振':'so',
    '犠打':'bant',
    '併殺打':'db_play',
}

In [6]:
df_batter = df_batter.rename(columns=name_j_to_e)

In [7]:
batter = df_batter.sample(1)

In [8]:
batter

,Team,背番号,name,ave,on_games,ab,pa,hit,hr,bp,...,walk,deadb,so,bant,db_play,長打率,出塁率,OPS,RC27,XR27
61,巨,5,セペダ,0,20,28,21,0,0,1,...,7,0,6,0,1,0,0.25,0.25,-0.40,0.09


In [9]:
for c in df_pitcher.columns:
    print c

Team
背番号
選手名
防御率
試合
勝利
敗北
セlブ
ホlルド
勝率
打者
投球回
被安打
被本塁打
与四球
与死球
奪三振
失点
自責点
WHIP
DIPS


In [10]:
pitcher_j_to_e = {
    '打者':'pa',
    '被安打':'hit',
    '被本塁打':'hr',
    '与死球':'deadb',
    '与四球':'walk',
    '奪三振':'so',
}
df_pitcher = df_pitcher.rename(columns=pitcher_j_to_e)

In [11]:
def calculate_ratio(df,pitcher=False):
    df_batter = df
    if pitcher:
        df_batter['contact'] = df['pa'] - df['walk'] - df['deadb'] - df['so']
    else:
        df_batter['contact'] = df.pa - (df.walk+df.deadb+df.bant+df.so)
    df_batter['not_contact'] = df_batter['pa'] - df_batter['contact']
    df_batter['contact_ratio'] = df_batter['contact'] / df_batter['pa']
    df_batter['contact_odds'] = df_batter['contact_ratio'] / (1-df_batter['contact_ratio'])

    df_batter['so_ratio'] = df_batter['so'] / df_batter['not_contact']
    df_batter['so_odds'] = df_batter['so_ratio'] / (1-df_batter['so_ratio'])
    
    df_batter['walk_ratio'] = (df_batter['walk'] + df_batter['deadb']) / df_batter['not_contact']
    df_batter['walk_odds'] = df_batter['walk_ratio'] / (1 - df_batter['walk_ratio'])
    
    df_batter['hr_ratio'] = df_batter['hr'] /df_batter['contact']
    df_batter['hr_odds'] = df_batter['hr_ratio'] / (1-df_batter['hr_ratio'])
    
    df_batter['on_field'] = df_batter['contact'] - df_batter['hr']
     
    df_batter['hit_ratio'] = df_batter['hit'] / df_batter['contact']
    df_batter['hit_odds'] = df_batter['hit_ratio']  / (1-df_batter['hit_ratio'])
    return df_batter

In [12]:
df_batter = calculate_ratio(df_batter)
df_pitcher = calculate_ratio(df_pitcher,pitcher=True)

In [60]:
df_pitcher[df_pitcher['pa'] > 100].sort('contact_odds',ascending=True).head(10)

,Team,背番号,選手名,防御率,試合,勝利,敗北,セlブ,ホlルド,勝率,...,contact_odds,so_ratio,so_odds,walk_ratio,walk_odds,hr_ratio,hr_odds,on_field,hit_ratio,hit_odds
8,広,70,ヒース,2.36,43,3,6,4,10,0.333,...,1.628205,0.756410,3.105263,0.243590,0.322034,0.023622,0.024194,124,0.338583,0.511905
173,ヤ,14,秋吉 亮,2.36,74,6,1,0,22,0.857,...,1.652174,0.704348,2.382353,0.295652,0.419753,0.031579,0.032609,184,0.268421,0.366906
65,阪,19,藤浪 晋太郎,2.40,28,14,7,0,0,0.667,...,1.675159,0.703822,2.376344,0.296178,0.420814,0.017110,0.017408,517,0.307985,0.445055
10,広,20,永川 勝浩,4.78,26,1,1,0,9,0.500,...,1.704545,0.613636,1.588235,0.386364,0.629630,0.026667,0.027397,73,0.360000,0.562500
110,中,41,浅尾 拓也,3.19,36,1,1,3,16,0.500,...,1.730769,0.653846,1.888889,0.346154,0.529412,0.033333,0.034483,87,0.344444,0.525424
140,横,19,山崎 康晃,1.92,58,2,4,37,7,0.333,...,1.756410,0.846154,5.500000,0.153846,0.181818,0.014599,0.014815,135,0.277372,0.383838
102,中,16,又吉 克樹,3.36,63,6,6,0,30,0.500,...,1.761468,0.752294,3.037037,0.247706,0.329268,0.026042,0.026738,187,0.333333,0.500000
74,阪,26,歳内 宏明,2.62,29,1,1,0,2,0.500,...,1.823529,0.450980,0.821429,0.549020,1.217391,0.010753,0.010870,92,0.161290,0.192308
146,横,65,国吉 佑樹,2.43,28,3,2,1,0,0.600,...,1.936170,0.595745,1.473684,0.404255,0.678571,0.010989,0.011111,90,0.241758,0.318841
37,巨,90,田口 麗斗,2.71,13,3,5,0,0,0.375,...,1.946237,0.688172,2.206897,0.311828,0.453125,0.027624,0.028409,176,0.298343,0.425197


In [61]:
df_pitcher[df_pitcher['pa']<100].

Team             広広広広広広広広広広巨巨巨巨巨巨巨巨阪阪阪阪阪阪阪阪阪阪阪阪阪阪阪阪阪中中中中中中中中中中中...
背番号                                                           2660
選手名              今井　啓介飯田　哲矢九里　亜蓮ザガースキー江草　仁貴中田　廉武内　久士中村　恭平西原　圭大河...
防御率                                                          496.4
試合                                                             453
勝利                                                              11
敗北                                                              25
セlブ                                                              1
ホlルド                                                            45
勝率                                                             9.5
pa                                                            2777
投球回                                                          593.8
hit                                                            685
hr                                                              65
walk                                                          

In [14]:
pitcher = df_pitcher[df_pitcher['pa'] > 100].sample(1)
batter = df_batter[df_batter['pa'] > 100].sample(1)

In [15]:
def odds_to_prob(odds):
    return odds / (1. + odds)

In [16]:
rdg = np.random.RandomState(71)

In [17]:
rdg.uniform()

0.1855752748185393

In [18]:
def match(batter,pitcher,parkfact=None):
    # contact or not
    contact_odds_ratio = batter['contact_odds'].values[0] * pitcher['contact_odds'].values[0]
    contact_prob = odds_to_prob(contact_odds_ratio)
    
    if rdg.uniform() < contact_prob:
        # consider hr
        hr_odds_ratio = batter['hr_odds'].values[0] * pitcher['hr_odds'].values[0]
        hr_prob = odds_to_prob(hr_odds_ratio)
        
        if rdg.uniform() < hr_prob:
            return 'hr',0
        
        hit_odds_ratio = batter['hit_odds'].values[0] * pitcher['hit_odds'].values[0]
        hit_ratio = odds_to_prob(hit_odds_ratio)
        
        if rdg.uniform() < hit_ratio:
            return 'hit',0
        else:
            return 'field_out',1
        
    else:
        # uncontact
        so_odds_ratio = batter['so_odds'].values[0] * pitcher['so_odds'].values[0]
        so_prob = odds_to_prob(so_odds_ratio)
        
        if rdg.uniform() < so_prob:
            return 'so',1
        else:
            return 'walk',0

In [ ]:
class Runners(object):
    def __init__(self):
        self.first = None
        self.second = None
        self.third = None
        
    def 
    
class StartingMembers(object):
    def __init__(self,members):
        self.length = len(members)
        self.members = members
        self.current = 1
        
    def nextbatter(self):
        if self.current == self.length:
            self.current = 1
        else:
            self.current += 1
        return self.members[self.current]
    
    

class GameStatus(object):
    def __init__(self,starting_batters,starting_pitchers):
        self.runners = Runners()
        self.out_count = 0
        self.inning = 1
        self.point = 0
        self.starting_batters = starting_batters
        self.starting_pitchers = starting_pitchers
        self.batting_order = 1
        
    def next_step(self):
        if self.out_count == 3:
            self.inning += 1
            self.out_count = 0
            return
        
        batter = self.starting_batters.nextbatter()
        pitcher = self.starting_pitchers
        
        result = match(batter=batter,pitcher=pitcher)
        

In [29]:
df_result = pd.DataFrame()
results= []
for i in range(1000):
    result_i,out = match(batter,pitcher)
#     print result_i,out
    results.append(result_i)
    
df_result['result'] = results

In [43]:
df_result['result'].unique()

array(['field_out', 'hit', 'so', 'walk'], dtype=object)

In [165]:
batter['contact_odds']

153    1.818182
Name: contact_odds, dtype: float64

In [169]:
pitcher['contact_odds'].values[0] * batter['contact_odds'].values[0]

3.662337662337662